In [1]:
#|export
"""
Run simply with
$ pytest
"""

import os
import subprocess
import numpy as np
import torch
import struct

from export import export_model, export_modelq8

file_path = "test/data/imagenette2/val_transformed/0/113"

In [2]:
#|export
# calculate reference values using Python model
model = torch.load("model.pt")
with open(file_path, "rb") as f:
    sizeof_float, nch, h, w = 4, 3, 224, 224
    image = torch.tensor(struct.unpack("f"*(nch*h*w), f.read(sizeof_float*nch*h*w))).view(1,nch,h,w)
    ref = model(image).detach()
    ref = torch.nn.functional.softmax(ref, dim=1).view(-1).numpy() # python model output

In [5]:
#|export
def execute(command):
    d = "test_outputs"
    os.makedirs(d, exist_ok=True)
    err = os.path.join(d, "err.txt")
    out = os.path.join(d, "stdout.txt")
    with open(err, mode="w") as fe:
        with open(out, mode="w") as fo:
            proc = subprocess.Popen(command, stdout=fo, stderr=fe)
            proc.wait()
    res = np.loadtxt(out)

    return res

def test_runfiles(quantized=True, file_path=file_path):
    """ test run.c and runq.c works with an acceptable tolerance """
    # run vanilla model in test mode
    export_model()
    command = ["./run", "model.bin", file_path]
    res = execute(command)
    assert np.allclose(res, ref, atol=1e-5, rtol=0), "run.c: Probabilities are not close."

    if quantized:
        # run quantized model test with group size = 1 and 2 in test mode
        export_modelq8(file_path="modelq8_1.bin", gs=1)
        resq1 = execute(["./runq", "modelq8_1.bin", file_path])
        assert np.allclose(resq1, ref, atol=1e-5, rtol=0), "runq.c (group size = 1): Probabilities are not close."

        export_modelq8(file_path="modelq8_2.bin", gs=2)
        resq2 = execute(["./runq", "modelq8_2.bin", file_path])
        assert np.allclose(resq2, ref, atol=1e-2, rtol=0), "runq.c (group size = 2):Probabilities are not close."

    print("Done")

In [6]:
test_runfiles()

wrote model.bin
Quantized (64, 3, 7, 7) to Q8_0 with max error 2.9802322387695312e-08
Quantized (64, 64, 3, 3) to Q8_0 with max error 2.9802322387695312e-08
Quantized (64, 64, 3, 3) to Q8_0 with max error 7.450580596923828e-09
Quantized (64, 64, 3, 3) to Q8_0 with max error 7.450580596923828e-09
Quantized (64, 64, 3, 3) to Q8_0 with max error 1.4901161193847656e-08
Quantized (128, 64, 3, 3) to Q8_0 with max error 1.4901161193847656e-08
Quantized (128, 128, 3, 3) to Q8_0 with max error 1.4901161193847656e-08
Quantized (128, 64, 1, 1) to Q8_0 with max error 2.9802322387695312e-08
Quantized (128, 128, 3, 3) to Q8_0 with max error 1.4901161193847656e-08
Quantized (128, 128, 3, 3) to Q8_0 with max error 1.4901161193847656e-08
Quantized (256, 128, 3, 3) to Q8_0 with max error 1.4901161193847656e-08
Quantized (256, 256, 3, 3) to Q8_0 with max error 1.4901161193847656e-08
Quantized (256, 128, 1, 1) to Q8_0 with max error 7.450580596923828e-09
Quantized (256, 256, 3, 3) to Q8_0 with max error 7